# Time series forecasting

This tutorial uses safeds on **US inflation rates data** to predict future inflation with a recurrent neural network. The **US inflation rates data** is a time series. A time series, is a series with frequently measured values. This time series measures the inflation of the US every Month from January 1947 until June 2023.


1. Load your data into a `Table`. The data is available under `docs/tutorials/data/US_Inflation_rates.csv`:


In [1]:
import os
print(os.getcwd())

ModuleNotFoundError: No module named 'safeds'

In [ ]:
from safeds.data.tabular.containers import Table

inflation = Table.from_csv_file("data/US_Inflation_rates.csv")
# For visualisation purposes we only print out the first 15 rows.
inflation.slice_rows(0,15)

This dataset contains two columns: date and value. The date column is right now still a string type with a format like this: "Year-Month-Day". We can convert it into a temporal type column like this:

In [2]:
inflation = inflation.replace_column("date", [inflation.get_column("date").from_str_to_temporal("%Y-%m-%d")])
inflation.slice_rows(0,15)

date,value
date,f64
1947-01-01,21.48
1947-02-01,21.62
1947-03-01,22.0
1947-04-01,22.0
1947-05-01,21.95
…,…
1947-11-01,23.06
1947-12-01,23.41
1948-01-01,23.68


As we can see the date column is no longer in quotes.

2. In the second step we prepare the data, to train our neural network. For that we need to normalize our data, because neural networks work better on values between 0 and 1. After that, we split the data into a training and test set.


In [3]:
from safeds.data.tabular.transformation import RangeScaler

scaler = RangeScaler()
trained_scaler, inflation = scaler.fit_and_transform(inflation, ["value"])
train_set, test_set = inflation.split_rows(percentage_in_first=0.8, shuffle=False)

3. To finish our data preparation we mark the datasets as time series, so they can be used in such context for the neural networks:

In [4]:
train_time_series = train_set.to_time_series_dataset(target_name="value", time_name="date")
test_time_series = test_set.to_time_series_dataset(target_name="value", time_name="date")


If we want to use a neural network to predict what the inflation will be next month, we have to provide it with a relevant chunk of past data that it can learn from. Think of it like looking at a series of snapshots from the past to guess what's going to happen next.

Instead of feeding the entire history of inflation data to the neural network all at once, which could be overwhelming and not very helpful, we divide the historical data into smaller, more manageable parts. This technique is known as "windowing."

A "window" in this context is just a set portion of consecutive data points from the time series. By giving the neural network these windows, it can start to understand how the numbers in the past relate to future values.

In our case, we want to look at the last 8 months of inflation data to make a guess about the inflation for the upcoming month. So, we choose a window size of 8. This window will slide across our time series data, providing the neural network with different 8-month segments to learn from.

Finally, we tell our neural network that our forecast horizon is 1, meaning that we're interested in predicting the value for just one step ahead—the next month's inflation rate.

So, simply put, we set up our neural network to take in 8-month chunks of inflation data to help it make a forecast about the inflation rate for the next month.



In [5]:
from safeds.ml.nn import NeuralNetworkRegressor

from safeds.ml.nn.converters import (
    InputConversionTimeSeries,
    OutputConversionTimeSeries,
)
from safeds.ml.nn.layers import (
    ForwardLayer,
    LSTMLayer,
)
window_size = 8

neural_network = NeuralNetworkRegressor(
    InputConversionTimeSeries(window_size=window_size, forecast_horizon=1),
    [ForwardLayer(input_size=window_size, output_size=256), LSTMLayer(input_size=256, output_size=1)],
    OutputConversionTimeSeries("predicted"),
)
fitted_neural_network = neural_network.fit(train_time_series)

C:\Users\ettel\PycharmProjects\Library\venv\Lib\site-packages\torch\utils\_device.py:78: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ..\aten\src\ATen\native\cudnn\RNN.cpp:1424.)
  return func(*args, **kwargs)


5. Now that we defined and trained our model, we can start making predictions.

In [6]:
prediction = fitted_neural_network.predict(
    test_time_series
)
prediction = prediction.to_table()
# For visualisation purposes we only print out the first 15 rows.
prediction.slice_rows(start=0, length=15)

date,value,predicted
date,f64,f64
2008-12-01,0.672607,0.627523
2009-01-01,0.674502,0.630725
2009-02-01,0.677236,0.632174
2009-03-01,0.676492,0.630603
2009-04-01,0.67725,0.628134
…,…,…
2009-10-01,0.690708,0.62319
2009-11-01,0.693276,0.623395
2009-12-01,0.693676,0.624262


6. Now we only need to inverse our predictions and we can start visualizing them:


In [7]:
trained_scaler._column_names.append("predicted")
prediction.inverse_transform_table(fitted_transformer=trained_scaler)


date,value,predicted
date,f64,f64
2008-12-01,211.398,0.627523
2009-01-01,211.933,0.630725
2009-02-01,212.705,0.632174
2009-03-01,212.495,0.630603
2009-04-01,212.709,0.628134
…,…,…
2023-02-01,301.648,0.682311
2023-03-01,301.808,0.682405
2023-04-01,302.918,0.682811
